In [ ]:
import pyodbc
from datetime import datetime
import os

class Bill:

    def __init__(self):
        try:
            self.conn = pyodbc.connect(
                "Driver={ODBC Driver 17 for SQL Server};"
                "Server=DESKTOP-LJ2060M\SQLEXPRESS;"
                "Database=bill_printing;"
                "Trusted_Connection=yes;"
            )
            self.cursor = self.conn.cursor()
            print("Database Connected Successfully\n")
        except:
            print("Connection Failed")
    
    def yes_no_input(self, message):
        while True:
            choice = input(message).lower()
            if choice in ['y', 'n']:
                return choice
            else:
                print("Please enter only 'y' or 'n'\n")

    def show_menu(self):
        print("\n------ MENU ------")
        self.cursor.execute("SELECT * FROM menu")
        rows = self.cursor.fetchall()

        for row in rows:
            print(row[0], row[1].strip(), "₹", row[2])
        print("------------------\n")

    def take_order(self):
        print("Start Taking Order (Enter 0 to Stop)\n")

        self.cursor.execute("SELECT item_id FROM menu")
        valid_ids = [row[0] for row in self.cursor.fetchall()]

        while True:
            try:
                item_id = int(input("Enter Item Number: "))

                if item_id == 0:
                    print("Order Taking Stopped\n")
                    break

                if item_id not in valid_ids:
                    print("Invalid Item Number Please select from menu\n")
                    continue

            except ValueError:
                print("Please enter numbers only\n")
                continue

            try:
                quantity = int(input("Enter Quantity: "))
                if quantity <= 0:
                    print("Quantity must be greater than 0\n")
                    continue
            except ValueError:
                print("Please enter valid quantity\n")
                continue

            self.cursor.execute("SELECT items FROM menu WHERE item_id=?", (item_id,))
            item = self.cursor.fetchone()
            item_name = item[0].strip()

            self.cursor.execute(
                "INSERT INTO orders (bill_id, item_id, quantity, order_date) VALUES (?, ?, ?, ?)",
                (self.bill_id, item_id, quantity, datetime.now())
            )
            self.conn.commit()

            print("Added:", item_name, "x", quantity, "\n")


    def generate_bill(self):

        self.cursor.execute("""
            SELECT m.items, o.quantity, m.price
            FROM orders o
            JOIN menu m ON o.item_id = m.item_id
            WHERE o.bill_id = ?
        """, (self.bill_id,))

        rows = self.cursor.fetchall()

        if not rows:
            print("No orders found\n")
            return 0

        bill_format = ""

        line = "-" * 50
        bill_format += line + "\n"
        bill_format += "|{:^48}|\n".format("Welcome to Hotel Chavishta")
        bill_format += line + "\n"
        bill_format += "| {:<3} {:<15} {:<8} {:>8} |\n".format("sr", "Menu", "qty", "price")
        bill_format += line + "\n"

        total_amount = 0
        sr = 1

        for row in rows:
            item = row[0].strip()
            qty = row[1]
            price = row[2]
            total = qty * price

            bill_format += "| {:<3} {:<15} {:<8} {:>8} |\n".format(sr, item, qty, total)

            total_amount += total
            sr += 1

        bill_format += line + "\n"
        bill_format += "| {:<18} {:>18} |\n".format("Total", total_amount)
        bill_format += line + "\n"

        print(bill_format)

        self.bill_format = bill_format
        self.total_amount = total_amount

        return total_amount

    def print_bill(self):

        choice = self.yes_no_input("Print Bill? (y/n): ")

        if choice == 'y':

            if not os.path.exists("Bills"):
                os.makedirs("Bills")

            file_path = f"Bills/Bill_{self.bill_id}.txt"

            with open(file_path, "w", encoding="utf-8") as f:
                f.write(self.bill_format)

            print("Bill Saved Successfully\n")
            print("Saved At:", file_path, "\n")

        else:
            print("Bill Not Printed\n")

    def run(self):

        print("Welcome to Hotel Chavishta\n")

        self.bill_id = datetime.now().strftime("%Y%m%d%H%M%S")

        self.show_menu()
        self.take_order()

        delivered = self.yes_no_input("Is Order Delivered? (y/n): ")

        if delivered != 'y':
            print("Order Not Delivered. Bill Cannot Be Generated")
            return

        generate = self.yes_no_input("Generate Bill? (y/n): ")

        if generate == 'y':
            total = self.generate_bill()
            if total > 0:
                self.print_bill()
        else:
            print("Bill Generation Cancelled")

system = Bill()
system.run()
